In [1]:
import os
import pandas as pd

os.chdir("../../")
from scripts.llm import get_num_of_tokens, get_completion
from scripts.utils import save_obj_as_pickle, read_obj_from_pickle
from scripts.data import make_database, make_prompts_for_clf

pd.set_option("display.max_colwidth", 150)

In [2]:
def loadData(dire):
    dsType = dire.split("/")[-1]
    fp1 = os.path.join(dire, f"{dsType}.data.txt")
    fp2 = os.path.join(dire, f"{dsType}.gold.txt")
    wic_data = pd.read_table(fp1, header=None, names=["word", "pos", "ix", "context1", "context2"])
    wic_label = pd.read_table(fp2, header=None, names=["label"])
    df = pd.concat([wic_data, wic_label], axis=1)
    df["text"] = "Target word: " + df["word"] + "\n" + "Context 1: " + df["context1"] + "\n" + "Context 2: " + df["context2"] + "\n"
    df = df.drop(columns=["word", "pos", "ix", "context1", "context2"])
    df.label = df.label.map({"T": "Yes", "F": "No"})
    df.reset_index(drop=True, inplace=True)

    return df


clf_task = "WiC"
test_dire = f"data/raw/text classification/{clf_task}/test"
test_df = loadData(test_dire)

dev_dire = f"data/raw/text classification/{clf_task}/dev"
dev_df = loadData(dev_dire)

test_df.head()

,label,text
0,Yes,Target word: defeat\nContext 1: It was a narrow defeat .\nContext 2: The army 's only defeat .\n
1,Yes,Target word: groom\nContext 1: Groom the dogs .\nContext 2: Sheila groomed the horse .\n
2,Yes,"Target word: penetration\nContext 1: The penetration of upper management by women .\nContext 2: Any penetration , however slight , is sufficient t..."
3,No,Target word: hit\nContext 1: We hit Detroit at one in the morning but kept driving through the night .\nContext 2: An interesting idea hit her .\n
4,No,Target word: deliberation\nContext 1: He was a man of judicial deliberation .\nContext 2: A little deliberation would have deterred them .\n


In [4]:
test_df.label.value_counts()

Yes    700
No     700
Name: label, dtype: int64

In [5]:
test_df.text.apply(get_num_of_tokens).describe()

count    1400.000000
mean       31.657857
std         6.606315
min        19.000000
25%        27.000000
50%        31.000000
75%        35.000000
max        68.000000
Name: text, dtype: float64

In [24]:
dev_df.label.value_counts()

No     319
Yes    319
Name: label, dtype: int64

In [25]:
dev_df.text.apply(get_num_of_tokens).describe()

count    638.000000
mean      31.547022
std        6.074897
min       19.000000
25%       27.000000
50%       31.000000
75%       35.000000
max       71.000000
Name: text, dtype: float64

### Database

Sources for the prompt data

In [26]:
database = make_database(test_df, dev_df, num_instance=500, max_instance_size=500)
database.keys()

dict_keys(['num_instance', 'max_instance_size', 'labels', 'testData', 'testInstances', 'devData', 'devInstances'])

#### Zero-shot

In [11]:
database["promptTemplates"] = dict()
database["promptTemplates"]["zero-shot"] = dict()

single_clf = "Analyze the usage of the given target word in the two subsequent contexts. " \
             "The target word may appear in various grammatical forms in each context. " \
             "Respond with 'Yes' if it maintains the same meaning across both contexts, and 'No' if it does not.\n\n" \
             "$text\nAnswer: "

batch_clf = "Analyze the usage of the following $num target words in the two contexts that immediately follow them. " \
            "These target words may appear in different grammatical forms across the two subsequent contexts. " \
            "Determine if each target word maintains the same meaning in the two subsequent contexts. " \
            "Provide your answers line by line, indicating 'Yes' if it does and 'No' if it does not.\n\n$texts\n" \
            "Answers:\n"

index_selection_one_cat_a_time = "Analyze the following $num target words and determine the index numbers of the target words where " \
                                 "the same meaning $be maintained across the two contexts that immediately follow them. " \
                                 "These target words may appear in different grammatical forms in each context.\n" \
                                 "If none of the target words satisfy this condition, write 'None.'.\n" \
                                 "If all the target words satisfy this condition, write 'All.'\n" \
                                 "Otherwise, provide the index numbers, each on a separate line.\n\n" \
                                 "Here are the target words along with their contexts:\n\n$texts\n" \
                                 "'None,' 'All,' or the index numbers of the target words where the same meaning $be maintained " \
                                 "in the two subsequent contexts are as follows:\n"

index_selection_one_cat_a_time_json =   "Analyze the following $num target words and determine the index numbers of the target words where " \
                                        "the same meaning $be maintained across the two contexts that immediately follow them. " \
                                        "These target words may appear in different grammatical forms in each context.\n" \
                                        "If none of the target words satisfy this condition, write 'None.'.\n" \
                                        "If all the target words satisfy this condition, write 'All.'\n" \
                                        "Otherwise, provide the index numbers.\n\n" \
                                        "Output your responses in JSON format with the key 'answer'.\nA formatted example output is provided below.\n" \
                                        "{'answer': [None/All or index numbers of the target words where the same meaning $be maintained in the two subsequent contexts]}\n\n" \
                                        "Here are the target words along with their contexts:\n\n$texts\n" \
                                        "JSON output:\n"

index_selection_all_cat_at_once = "Analyze the following $num target words, which may appear in different grammatical forms in the two subsequent contexts. " \
                                  "First, list the index numbers of target words that maintain the same meaning in the two subsequent contexts. " \
                                  "Then, list the index numbers of target words that do not maintain the same meaning in the two subsequent contexts.\n" \
                                  "If none of the target words satisfy a condition, write 'None.'\nIf all the target words satisfy a condition, write 'All.'\n" \
                                  "Otherwise, provide the index numbers of the target words that satisfy each condition.\n\n" \
                                  "Here are the target words along with their contexts:\n\n$texts\n" \
                                  "Output your responses in JSON format with two keys: 'yes' for target words used with consistent meanings and " \
                                  "'no' for those used with inconsistent meanings.\nA formatted example output is provided below.\n" \
                                  "{'yes': [None/All or index numbers of target words used with consistent meanings], " \
                                  "'no': [None/All or index numbers of target words used with inconsistent meanings]}"

index_selection_all_cat_at_once_adjusted =  "Analyze the following $num target words, which may appear in different grammatical forms in the two subsequent contexts. " \
                                            "First, list the index numbers of target words that maintain the same meaning in the two subsequent contexts. " \
                                            "Then, list the index numbers of target words that do not maintain the same meaning in the two subsequent contexts.\n" \
                                            "If none of the target words satisfy a condition, write 'None.'\nIf all the target words satisfy a condition, write 'All.'\n" \
                                            "Otherwise, provide the index numbers of the target words that satisfy each condition.\n\n" \
                                            "Output your responses in JSON format with two keys: 'yes' for target words used with consistent meanings and " \
                                            "'no' for those used with inconsistent meanings.\nA formatted example output is provided below.\n" \
                                            "{'yes': [None/All or index numbers of target words used with consistent meanings], " \
                                            "'no': [None/All or index numbers of target words used with inconsistent meanings]}\n\n" \
                                            "Here are the target words along with their contexts:\n\n$texts\n" \
                                            "JSON output:\n"

tasks = ["single_clf", "batch_clf", "index_selection_one_cat_a_time", 
         "index_selection_all_cat_at_once", "index_selection_all_cat_at_once_adjusted", "index_selection_one_cat_a_time_json"]
promptTemplates = [single_clf, batch_clf, index_selection_one_cat_a_time, 
                   index_selection_all_cat_at_once, index_selection_all_cat_at_once_adjusted, index_selection_one_cat_a_time_json]

for task, tmp in zip(tasks, promptTemplates):
    database["promptTemplates"]["zero-shot"][task] = tmp

In [16]:
os.makedirs("data/databases/text classification/", exist_ok=True)
save_obj_as_pickle(database, f"data/databases/text classification/{clf_task}.pkl")

Saved object to data/databases/text classification/WiC.pkl


#### Test Prompts

- The main purpose is to check if LLMs can output the desired formats given the prompts 

In [9]:
dev = []
num_instance = 2
label_attr_converter = lambda t: {"Yes": "is", "No": "isn't"}[t]
taskSizes = [3, 5]
for propmtMode in ["zero-shot"]:
    for task in tasks:
        if task == "single_clf":
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode)[:num_instance])
            continue

        for taskSize in taskSizes:  
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode, taskSize, attr="be", 
                                            label_attr_converter=label_attr_converter, num_instance=num_instance))

dev = pd.concat(dev).reset_index(drop=True)

In [10]:
for p in dev[(dev["taskSize"] <= 3) & (dev["taskIndex"] == 1)].prompt:
    print(p)
    print("-"*50)
    print()

Analyze the usage of the given target word in the two subsequent contexts. The target word may appear in various grammatical forms in each context. Respond with 'Yes' if it maintains the same meaning across both contexts, and 'No' if it does not.

Target word: board
Context 1: Room and board .
Context 2: He nailed boards across the windows .

Answer: 
--------------------------------------------------

Analyze the usage of the following 3 target words in the two contexts that immediately follow them. These target words may appear in different grammatical forms across the two subsequent contexts. Determine if each target word maintains the same meaning in the two subsequent contexts. Provide your answers line by line, indicating 'Yes' if it does and 'No' if it does not.

1. Target word: post
Context 1: Post a warning at the dump .
Context 2: The newspaper posted him in Timbuktu .

2. Target word: terror
Context 1: He used terror to make them confess .
Context 2: He was the terror of the

In [11]:
dev["preds"] = dev.prompt.apply(get_completion)
dev

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,preds
0,1,Analyze the usage of the given target word in the two subsequent contexts. The target word may appear in various grammatical forms in each context...,No,NA,single_clf,0,False,1,No
1,2,Analyze the usage of the given target word in the two subsequent contexts. The target word may appear in various grammatical forms in each context...,No,NA,single_clf,0,False,1,Yes
2,1,Analyze the usage of the following 3 target words in the two contexts that immediately follow them. These target words may appear in different gra...,"[No, No, No]",NA,batch_clf,0,False,3,1. No\n2. Yes\n3. No
3,2,Analyze the usage of the following 3 target words in the two contexts that immediately follow them. These target words may appear in different gra...,"[Yes, Yes, No]",NA,batch_clf,0,False,3,1. Yes\n2. Yes\n3. Yes
4,1,Analyze the usage of the following 5 target words in the two contexts that immediately follow them. These target words may appear in different gra...,"[No, No, No, Yes, No]",NA,batch_clf,0,False,5,1. Yes\n2. No\n3. No\n4. Yes\n5. Yes
5,2,Analyze the usage of the following 5 target words in the two contexts that immediately follow them. These target words may appear in different gra...,"[Yes, Yes, No, Yes, No]",NA,batch_clf,0,False,5,1. No\n2. Yes\n3. Yes\n4. Yes\n5. Yes
6,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,{All},No,index_selection_one_cat_a_time,0,False,3,None.
7,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,{None},Yes,index_selection_one_cat_a_time,0,False,3,None.
8,2,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,{3},No,index_selection_one_cat_a_time,0,False,3,None.
9,2,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,"{1, 2}",Yes,index_selection_one_cat_a_time,0,False,3,All


### Adjusted

In [13]:
dev = []
num_instance = 2
label_attr_converter = lambda t: {"Yes": "is", "No": "isn't"}[t]

taskSizes = [3, 5]
for propmtMode in ["zero-shot"]:
    for task in tasks[-2:]:
        if task == "single_clf":
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode)[:num_instance])
            continue

        for taskSize in taskSizes:  
            dev.append(make_prompts_for_clf(database, task, "dev", propmtMode, taskSize, attr="be", 
                                            label_attr_converter=label_attr_converter, num_instance=num_instance))

dev = pd.concat(dev).reset_index(drop=True)

In [14]:
for p in dev[(dev["taskSize"] <= 5) & (dev["taskIndex"] == 1)].prompt:
    print(p)
    print("-"*50)
    print()

Analyze the following 3 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index numbers of target words that maintain the same meaning in the two subsequent contexts. Then, list the index numbers of target words that do not maintain the same meaning in the two subsequent contexts.
If none of the target words satisfy a condition, write 'None.'
If all the target words satisfy a condition, write 'All.'
Otherwise, provide the index numbers of the target words that satisfy each condition.

Output your responses in JSON format with two keys: 'yes' for target words used with consistent meanings and 'no' for those used with inconsistent meanings.
A formatted example output is provided below.
{'yes': [None/All or index numbers of target words used with consistent meanings], 'no': [None/All or index numbers of target words used with inconsistent meanings]}

Here are the target words along with their contexts:

1. Target word: post
Conte

In [15]:
model = "meta-llama/Llama-3-8b-chat-hf"
dev["preds"] = dev.prompt.apply(lambda p: get_completion(p, model))
dev

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,preds
0,1,"Analyze the following 3 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index number...","{'no': {'All'}, 'yes': {'None'}}",NA,index_selection_all_cat_at_once_adjusted,0,False,3,"After analyzing the target words, I found the following results:\n\n{\n""yes"": [2],\n""no"": [1, 3]\n}\n\nHere's the explanation:\n\n* Target word 2 ..."
1,2,"Analyze the following 3 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index number...","{'no': {3}, 'yes': {1, 2}}",NA,index_selection_all_cat_at_once_adjusted,0,False,3,"After analyzing the target words, I found the following results:\n\n{\n""yes"": [2],\n""no"": [1, 3]\n}\n\nExplanation:\n\n* Target word 2 (""abridge"")..."
2,1,"Analyze the following 5 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index number...","{'no': {1, 2, 3, 5}, 'yes': {4}}",NA,index_selection_all_cat_at_once_adjusted,0,False,5,"After analyzing the target words, I found the following results:\n\n{\n""yes"": [5],\n""No"": [1, 2, 3, 4]\n}\n\nExplanation:\n\n* Target word 5 ""peak..."
3,2,"Analyze the following 5 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index number...","{'no': {3, 5}, 'yes': {1, 2, 4}}",NA,index_selection_all_cat_at_once_adjusted,0,False,5,"Here is the analysis of the target words:\n\n```\n{\n ""yes"": [2], \n ""no"": [1, 3, 4, 5]\n}\n```\n\nExplanation:\n\n* Target word 2 (""abridge..."
4,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,{All},No,index_selection_one_cat_a_time_json,0,False,3,"Here is the analysis:\n\nAfter analyzing the target words and their contexts, I found that the same meaning isn't maintained across the two contex..."
5,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,{None},Yes,index_selection_one_cat_a_time_json,0,False,3,Here is the analysis of the target words:\n\n1. Target word: post\nContext 1: Post a warning at the dump .\nContext 2: The newspaper posted him in...
6,2,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,{3},No,index_selection_one_cat_a_time_json,0,False,3,Here is the analysis of the target words:\n\n1. Target word: amuse\nContext 1: I watch these movies because they amuse me .\nContext 2: The clown ...
7,2,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,"{1, 2}",Yes,index_selection_one_cat_a_time_json,0,False,3,Here is the analysis of the target words:\n\n1. Target word: amuse\nContext 1: I watch these movies because they amuse me .\nContext 2: The clown ...
8,1,Analyze the following 5 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,"{1, 2, 3, 5}",No,index_selection_one_cat_a_time_json,0,False,5,Here is the analysis of the target words:\n\n1. Target word: post\nContext 1: Post a warning at the dump .\nContext 2: The newspaper posted him in...
9,1,Analyze the following 5 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,{4},Yes,index_selection_one_cat_a_time_json,0,False,5,Here is the analysis of the target words:\n\n1. Target word: post\nContext 1: Post a warning at the dump .\nContext 2: The newspaper posted him in...


### Make prompts

In [20]:
database = read_obj_from_pickle(f"data/databases/text classification/{clf_task}.pkl")

Read object from data/databases/text classification/WiC.pkl


In [13]:
# out = []
# num_instance = 100

# taskSizes = [3, 5, 10, 20, 50]
# for propmtMode in ["zero-shot"]:
#     for task in tasks:

#         if task == "single_clf":
#             out.append(make_prompts_for_clf(database, task, "test", propmtMode))
#             continue

#         for taskSize in taskSizes:
#             out.append(make_prompts_for_clf(database, task, "test", propmtMode, taskSize, attr="be", 
#                                             label_attr_converter=label_attr_converter, num_instance=num_instance))

# out = pd.concat(out)
# out.reset_index(drop=True, inplace=True)

In [14]:
# os.makedirs("results/text classification/", exist_ok=True)
# out.to_json(f"results/text classification/{clf_task}.json", orient="records", lines=True)

In [24]:
fp = f"results/text classification/{clf_task}.json"
out = [pd.read_json(fp, lines=True)]

num_instance = 100
taskSizes = [3, 5, 10, 20, 50]
for propmtMode in ["zero-shot"]:
    for taskSize in taskSizes:
        for task in tasks[-2:]:
            out.append(make_prompts_for_clf(database, task, "test", propmtMode, taskSize, attr="be", 
                                            label_attr_converter=label_attr_converter, num_instance=num_instance))

out = pd.concat(out)
out.reset_index(drop=True, inplace=True)

os.makedirs("results/text classification/", exist_ok=True)
out.to_json(f"results/text classification/{clf_task}.json", orient="records", lines=True)                                             

In [25]:
out.task.value_counts()

single_clf                                  1400
index_selection_one_cat_a_time              1000
index_selection_one_cat_a_time_json         1000
batch_clf                                    500
index_selection_all_cat_at_once              500
index_selection_all_cat_at_once_adjusted     500
Name: task, dtype: int64

In [26]:
out.prompt.apply(get_num_of_tokens).describe()

count    4900.000000
mean      552.795714
std       574.553125
min        75.000000
25%        95.000000
50%       312.000000
75%       785.000000
max      2017.000000
Name: prompt, dtype: float64

In [27]:
sub = out.copy()[(out.taskIndex == 1) & (out.taskSize <= 3)]
sub

,taskIndex,prompt,answer,targetLabel,task,#shot,CoT,taskSize,gpt-3.5-turbo-0125-completion,mistralai/Mixtral-8x7B-Instruct-v0.1-completion,meta-llama/Llama-3-8b-chat-hf-completion,meta-llama/Llama-3-70b-chat-hf-completion,gpt-4-turbo-2024-04-09-completion,mistralai/Mistral-7B-Instruct-v0.2-completion,lmsys/vicuna-13b-v1.5-completion
0,1,Analyze the usage of the given target word in the two subsequent contexts. The target word may appear in various grammatical forms in each context...,Yes,NA,single_clf,0,False,1,Yes,"Yes, the word ""defeat"" maintains the same meaning across both contexts. In Context 1, ""defeat"" is used to describe a narrow loss in a competition...","The target word ""defeat"" appears in different grammatical forms in each context:\n\n* In Context 1, ""defeat"" is a noun, referring to the outcome o...",Yes,Yes,"Yes, in both contexts, the word ""defeat"" maintains the same meaning, which is an unsuccessful outcome or loss.",Yes.
1400,1,Analyze the usage of the following 3 target words in the two contexts that immediately follow them. These target words may appear in different gra...,"[No, No, Yes]",NA,batch_clf,0,False,3,1. No\n2. Yes\n3. Yes,"1. Edit\nContext 1: Edit (verb) - to prepare (something, such as a film or a literary work) for publication or presentation by correcting, revisi...",Here are the analyses for each target word:\n\n1. Target word: edit\nContext 1: Edit film .\nContext 2: The same family has been editing the influ...,Here are the analyses:\n\n1. Target word: edit\nContext 1: Edit film .\nContext 2: The same family has been editing the influential newspaper for ...,1. Edit\n - Context 1: Edit film.\n - Context 2: The same family has been editing the influential newspaper for almost 100 years.\n - Answer...,"1. Target word: edit\nAnswer: No.\nIn Context 1, ""edit"" refers to making changes to a film.\nIn Context 2, ""edit"" refers to making changes to a n...",1. Yes\n2. Yes\n3. Yes
1900,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning isn't maintained across the two co...,"[1, 2]",No,index_selection_one_cat_a_time,0,False,3,None.,1.\nContext 1: Edit film .\n(film: a series of moving pictures telling a story and presented on a screen)\n\nContext 2: The same family has been ...,"After analyzing the target words and their contexts, I found that:\n\n* The target word ""edit"" has a different meaning in Context 1 (editing film)...",1,1. Target word: edit\n - Context 1: Edit film.\n - Context 2: The same family has been editing the influential newspaper for almost 100 years....,"The target word ""excursion"" does not maintain the same meaning in the two contexts. In Context 1, it refers to a trip or an outing, while in Cont...","None.\n\nThe target words ""edit,"" ""vision,"" and ""excursion"" maintain the same meaning across the two contexts in which they appear."
1901,1,Analyze the following 3 target words and determine the index numbers of the target words where the same meaning is maintained across the two conte...,[3],Yes,index_selection_one_cat_a_time,0,False,3,None.,"1\n2\n\nExplanation:\n\n1. The word ""edit"" in Context 1 refers to making changes or modifications to a film, while in Context 2, it refers to mak...","After analyzing the target words and their contexts, I found that:\n\n* The target word ""edit"" does not maintain the same meaning across the two c...",2\n3,2,"None.\n\nThe meaning of the target word ""edit"" in Context 1 is related to making changes to a film, while in Context 2, it refers to making chang...","1. Target word: edit\nIndex numbers: 1, 2\n2. Target word: vision\nIndex numbers: 1, 2\n3. Target word: excursion\nIndex numbers: 1, 2\n\nTherefo..."
2900,1,"Analyze the following 3 target words, which may appear in different grammatical forms in the two subsequent contexts. First, list the index number...","{'no': [1, 2], 'yes': [3]}",NA,index_selection_all_cat_at_once,0,False,3,"{\n ""yes"": [\n 2\n ],\n

In [29]:
out["lmsys/vicuna-13b-v1.5-completion"].str.contains("TOO_").sum()

1